:::{.content-hidden}
# Visual Exploration
:::


Unzip the image files, e.g. from your Drive.

In [ ]:
!zip -r /content/drive/MyDrive/2024-01-09-Bauernproteste/2024-01-09-Images-Clean.zip media

Import the CSV-File exported from Gephi. Set `sample_size` to your desired number, I recommend a low number, e.g. 5.

In [54]:
import pandas as pd

gephi_file = "/content/drive/MyDrive/2024-01-09-Bauernproteste/2024-01-11-Google-Vision-Graph-w-modclasses.csv"  #@param {type:"string"}
sample_size = 5 
gephi_df = pd.read_csv(gephi_file)

Render the Sample: Hit run for the next cell to create an HTML view of image classifications. The HTML will also be saved to file, check the files in the left pane for a file named `{formatted_date}-Gephi-Mod-Classes-Visualisation.html` to download the document to your computer. The file includes the `base64` encoded images.

In [ ]:
# See linked notebook for code.